# Sequential Labelling

## Import Modules

In [27]:
import evaluate
import numpy as np
import transformers
import tensorflow as tf

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification, create_optimizer, TFAutoModelForTokenClassification
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

## Import Dataset

In [3]:
nergrit = load_dataset('id_nergrit_corpus', 'ner')

Generating validation split: 100%|██████████| 2521/2521 [00:00<00:00, 5879.31 examples/s]


In [4]:
nergrit

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 12532
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2399
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2521
    })
})

In [5]:
nergrit['train'][0]

{'id': '0',
 'tokens': ['Indonesia',
  'mengekspor',
  'produk',
  'industri',
  'skala',
  'besar',
  'ke',
  'Amerika',
  'Serikat',
  '.',
  'Ekspor',
  'dilakukan',
  'melalui',
  'Pelabuhan',
  'Tanjung',
  'Priok',
  ',',
  'Jakarta',
  'Utara',
  ',',
  'Selasa',
  '(',
  '15',
  '/',
  '8',
  '/',
  '2018',
  ')',
  '.',
  'Komoditas',
  'yang',
  'dikirim',
  'terdiri',
  'dari',
  '50',
  'persen',
  'sepatu',
  ',',
  '15',
  'persen',
  'garmen',
  ',',
  '10',
  'persen',
  'produk',
  'karet',
  ',',
  'ban',
  'dan',
  'turunannya',
  ',',
  'alat',
  '-',
  'alat',
  'elektronik',
  '10',
  'persen',
  ',',
  'dan',
  'produk',
  'lainnya',
  '15',
  'persen',
  '.',
  '"',
  'Bukan',
  'bahan',
  'mentah',
  ',',
  'tetapi',
  'sudah',
  'bahan',
  '-',
  'bahan',
  'produksi',
  ',',
  'produk',
  '-',
  'produk',
  'industri',
  'yang',
  'kita',
  'harapkan',
  'ini',
  'akan',
  'meningkatkan',
  'ekspor',
  'kita',
  ',',
  '"',
  'kata',
  'Presiden',
  'dalam',


In [6]:
label_list = nergrit["train"].features[f"ner_tags"].feature.names
label_list

['B-CRD',
 'B-DAT',
 'B-EVT',
 'B-FAC',
 'B-GPE',
 'B-LAN',
 'B-LAW',
 'B-LOC',
 'B-MON',
 'B-NOR',
 'B-ORD',
 'B-ORG',
 'B-PER',
 'B-PRC',
 'B-PRD',
 'B-QTY',
 'B-REG',
 'B-TIM',
 'B-WOA',
 'I-CRD',
 'I-DAT',
 'I-EVT',
 'I-FAC',
 'I-GPE',
 'I-LAN',
 'I-LAW',
 'I-LOC',
 'I-MON',
 'I-NOR',
 'I-ORD',
 'I-ORG',
 'I-PER',
 'I-PRC',
 'I-PRD',
 'I-QTY',
 'I-REG',
 'I-TIM',
 'I-WOA',
 'O']

## Preprocessing

In [8]:
# use IndoBERT
tokenizer = AutoTokenizer.from_pretrained('indolem/indobert-base-uncased')

In [9]:
example = nergrit["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'indonesia',
 'mengekspor',
 'produk',
 'industri',
 'skala',
 'besar',
 'ke',
 'amerika',
 'serikat',
 '.',
 'ekspor',
 'dilakukan',
 'melalui',
 'pelabuhan',
 'tanjung',
 'priok',
 ',',
 'jakarta',
 'utara',
 ',',
 'selasa',
 '(',
 '15',
 '/',
 '8',
 '/',
 '2018',
 ')',
 '.',
 'komoditas',
 'yang',
 'dikirim',
 'terdiri',
 'dari',
 '50',
 'persen',
 'sepatu',
 ',',
 '15',
 'persen',
 'garmen',
 ',',
 '10',
 'persen',
 'produk',
 'karet',
 ',',
 'ban',
 'dan',
 'turunan',
 '##nya',
 ',',
 'alat',
 '-',
 'alat',
 'elektronik',
 '10',
 'persen',
 ',',
 'dan',
 'produk',
 'lainnya',
 '15',
 'persen',
 '.',
 '"',
 'bukan',
 'bahan',
 'mentah',
 ',',
 'tetapi',
 'sudah',
 'bahan',
 '-',
 'bahan',
 'produksi',
 ',',
 'produk',
 '-',
 'produk',
 'industri',
 'yang',
 'kita',
 'harapkan',
 'ini',
 'akan',
 'meningkatkan',
 'ekspor',
 'kita',
 ',',
 '"',
 'kata',
 'presiden',
 'dalam',
 'sambutannya',
 'pada',
 'acara',
 'pelepasan',
 'ekspor',
 'di',
 'jakarta',
 'international',
 

Based on the documentation:

> This adds some special tokens [CLS] and [SEP] and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the word_ids method.
2. Assigning the label -100 to the special tokens [CLS] and [SEP] so they're ignored by the PyTorch loss function.
3. Only labeling the first token of a given word. Assign -100 to other subtokens from the same word.

So it is needed to realign the token and labels, and truncate the sequence if it is longer than the models maximum length

In [10]:
def tokenize_and_align_labels(dataset):
    tokenized_inputs = tokenizer(dataset["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(dataset[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_nergrit = nergrit.map(tokenize_and_align_labels, batched=True) #processing in batch

Map: 100%|██████████| 2521/2521 [00:00<00:00, 7418.86 examples/s]


Based on the reference:

> It's more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

So, creating the data collator.

In [12]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

## Metrics Evaluation

I will be using just accuracy

In [13]:
seqeval = evaluate.load("seqeval")

In [14]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "accuracy": results["overall_accuracy"],
    }

## Fine-Tuning the Model

In [18]:
labels = [
    "B-CRD", "B-DAT", "B-EVT", "B-FAC", "B-GPE", "B-LAN", "B-LAW", "B-LOC", "B-MON", "B-NOR", 
    "B-ORD", "B-ORG", "B-PER", "B-PRC", "B-PRD", "B-QTY", "B-REG", "B-TIM", "B-WOA",
    "I-CRD", "I-DAT", "I-EVT", "I-FAC", "I-GPE", "I-LAN", "I-LAW", "I-LOC", "I-MON", "I-NOR",
    "I-ORD", "I-ORG", "I-PER", "I-PRC", "I-PRD", "I-QTY", "I-REG", "I-TIM", "I-WOA", "O",
]

id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}


In [19]:
len(labels)

39

In [20]:
batch_size = 16
num_train_epochs = 3
num_train_steps = (len(tokenized_nergrit["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [23]:
model = TFAutoModelForTokenClassification.from_pretrained(
    "indolem/indobert-base-uncased", num_labels=39, id2label=id2label, label2id=label2id, from_pt=True
)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_nergrit["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_nergrit["validation"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [25]:
model.compile(optimizer=optimizer)

In [29]:
# create the callaback for the model
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

push_to_hub_callback = PushToHubCallback(
    output_dir="indobert-base-uncased-finetuned-nergrit",
    tokenizer=tokenizer,
)

callbacks = [metric_callback, push_to_hub_callback]

Cloning https://huggingface.co/apwic/indobert-base-uncased-finetuned-nergrit into local empty directory.


In [30]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3
 33/783 [>.............................] - ETA: 39:04 - loss: 1.8538